In [1]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, roc_auc_score, recall_score
import re
from plotly.subplots import make_subplots

In [2]:
df_llm = pd.read_csv('../datasets/dessi-mf/dessi-mf_gpt/test.csv')
labels_llm = pd.read_csv('../datasets/dessi-mf/dessi-mf_gpt/test_labels_pii.csv')
classes_llm = pd.read_csv('../datasets/dessi-mf/dessi-mf_gpt/test_classes.csv')
multiclass_llm = pd.read_csv('../datasets/dessi-mf/dessi-mf_gpt/test_labels_multiclass.csv')
dataset_llm = pd.read_csv('../datasets/dessi-mf/dessi-mf_gpt/test_dataset.csv')

test_lan = pd.read_csv("../datasets/test_languages/test.csv")
test_lan_pii = pd.read_csv("../datasets/test_languages/test_labels_pii.csv")
test_lan_classes = pd.read_csv("../datasets/test_languages/test_classes.csv")
test_lan_datasets = pd.read_csv("../datasets/test_languages/test_dataset.csv")

kaggle_data = pd.read_csv('../datasets/kaggle_datasets/all_datasets.csv')
kaggle_labels = pd.read_csv('../datasets/kaggle_datasets/all_datasets_labels_pii.csv')
kaggle_dataset = pd.read_csv('../datasets/kaggle_datasets/all_datasets_names.csv')

openml_data = pd.read_csv('../datasets/openml_datasets/all_datasets.csv')
openml_labels = pd.read_csv('../datasets/openml_datasets/all_datasets_labels_pii.csv')
openml_dataset = pd.read_csv('../datasets/openml_datasets/all_datasets_names.csv')

openml_2_data = pd.read_csv('../datasets/openml_datasets_2/all_datasets.csv')
openml_2_labels = pd.read_csv('../datasets/openml_datasets_2/all_datasets_labels_pii.csv')
openml_2_dataset = pd.read_csv('../datasets/openml_datasets_2/all_datasets_names.csv')

In [3]:
def load_predictions(path):
    with open(path, "r") as f:
        response_text = f.read()
    responses = response_text.split("\n")[1:]
    detected_classes = []
    for j in range(len(responses)):
        match1 = re.search(f"('detected_classes'|\"detected_classes\"|detected_classes): \[(.*?)\]", responses[j])
        detected = match1.group(2).replace("'", "").replace("\"", "").replace(", ", ",")
        if "," in detected:
            detected = sorted(detected.split(","))
            detected = ",".join(detected)
        detected_classes.append(detected)
    pii_classes = ["credit_card_number", "email", "full_address", "full_name", "iban", "id_card", "longitude_and_latitude", "national_identification_number", 
                    "passport_number", "phone_number"]

    pii_transformed = []
    for i in range(len(detected_classes)):
        if any(a in detected_classes[i] for a in pii_classes):                       
            pii_transformed.append("pii")
        else:
            pii_transformed.append("non-pii")
    return pii_transformed, detected_classes


In [4]:
predictions_own_dataset, detected_classes_own_dataset = load_predictions("gpt_predictions/dessi-mf_results.txt")

In [5]:
results_own_dataset_pii = pd.DataFrame({
    "Column": df_llm.columns,
    "True Label": labels_llm["label"],
    "Prediction": predictions_own_dataset,
    "Classes": classes_llm["class"],
    "Predicted Class": detected_classes_own_dataset,
    "Dataset": dataset_llm["dataset"]
})

In [6]:
def plot_gpt_results(results_pii, title):
    y_true = results_pii["True Label"].values
    y_pred = results_pii["Prediction"].values
    y_true_bin = [1 if label == "pii" else 0 for label in y_true]
    y_pred_bin = [1 if label == "pii" else 0 for label in y_pred]

    cm = confusion_matrix(y_true, y_pred, labels=["pii", "non-pii"])
    cm_norm = confusion_matrix(y_true, y_pred, labels=["pii", "non-pii"], normalize="true")

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="weighted")
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    auc_roc = roc_auc_score(y_true_bin, y_pred_bin)

    fig = make_subplots(
        rows=1, cols=2, 
        subplot_titles=["Evaluation Metrics", "Confusion Matrix"]
    )

    metrics = {
        "weighted Precision": precision,
        "weighted Recall": recall,
        "weighted F1 Score": f1,
        "Accuracy": accuracy,
        "AUC-ROC": auc_roc
    }

    fig.add_trace(go.Bar(
        x=list(metrics.keys()),
        y=list(metrics.values()), 
        showlegend=False
    ), row=1, col=1)

    text_values = [
        [f"{cm[0,0]}<br>{cm_norm[0,0]:.2f}", f"{cm[0,1]}<br>{cm_norm[0,1]:.2f}"],
        [f"{cm[1,0]}<br>{cm_norm[1,0]:.2f}", f"{cm[1,1]}<br>{cm_norm[1,1]:.2f}"]
    ]
    cm_heatmap = go.Heatmap(
        z=cm, 
        x=["Pred: pii", "Pred: non-pii"], 
        y=["True: pii", "True: non-pii"], 
        colorscale="Blues", 
        showscale=False,
        text=text_values, 
        texttemplate="%{text}",
        textfont={"size":20},
    )

    fig.add_trace(cm_heatmap, row=1, col=2)

    fig.update_layout(
        title=f"Confusion Matrix and Evaluation Metrics for {title}",
        height=500, 
        width=1000,
        showlegend=False
    )

    fig.show()
plot_gpt_results(results_own_dataset_pii, "GPT Prediction on dessi-mf")

In [7]:
#False negatives
results_own_dataset_pii.loc[(results_own_dataset_pii["True Label"] == "pii") & (results_own_dataset_pii["Prediction"] == "non-pii")]

,Column,True Label,Prediction,Classes,Predicted Class,Dataset
49,ypqnvPf7C,pii,non-pii,ID_Card,,dessi
216,fUF0eWI,pii,non-pii,ssn_fr_FR,,faker
294,pBLzHfz3,pii,non-pii,passport_number_en,,faker
306,N9Vg8W,pii,non-pii,passport_number_mixed,,faker
367,VJvwSDrtqohUmuhVI8,pii,non-pii,credit_card_number_mixed,,mimesis
514,cosTsS3DeRo7qY,pii,non-pii,passport_number_fr_FR,,faker
522,RhALtY5U3SObMEq,pii,non-pii,passport_number_de_DE,,faker


All false negatives have corrupted column names and have except the address number values

In [8]:
#False positives
results_own_dataset_pii.loc[(results_own_dataset_pii["True Label"] == "non-pii") & (results_own_dataset_pii["Prediction"] == "pii")]

,Column,True Label,Prediction,Classes,Predicted Class,Dataset
275,155sXZoMZwoOmNqRWUXQ,non-pii,pii,ean_de_DE,credit_card_number,faker


In [9]:
CLASS_NAMES = results_own_dataset_pii["Classes"].unique()
CLASS_NAMES = [a.lower().split("_")[0] for a in CLASS_NAMES]

def is_valid_word(word):
    return word.lower().split("_")[0] in CLASS_NAMES

def create_analysis_plot(results_df, title, dataset="own_data"):
    cla_new, lan = [], []
    for i in range(len(results_df["Classes"])):
        if "mixed" in results_df.iloc[i, 3] or "de_DE" in results_df.iloc[i, 3] or "fr_FR" in results_df.iloc[i, 3]:
            cla_new.append(results_df.iloc[i, 3][:-6])
        elif "_en" in results_df.iloc[i, 3] or "_de" in results_df.iloc[i, 3] or "_fr" in results_df.iloc[i, 3] or "_it" in results_df.iloc[i,3
                                                                        ] or "_zh" in results_df.iloc[i,3]:
            cla_new.append(results_df.iloc[i, 3][:-3])
        else:
            cla_new.append(results_df.iloc[i, 3])
            
        if "mixed" in results_df.iloc[i, 3]:
            lan.append("mixed language")
        elif "_fr" == results_df.iloc[i,3][-3:].lower():
            lan.append("french")
        elif "_it" == results_df.iloc[i,3][-3:].lower():
            lan.append("italian")
        elif "_zh" == results_df.iloc[i,3][-3:].lower():
            lan.append("chinese")
        elif "_de" == results_df.iloc[i,3][-3:].lower():
            lan.append("german")
        elif "_en" == results_df.iloc[i,3][-3:].lower():
            lan.append("english")
        else:
            lan.append("dessi data")
    results_df["Classes_new"] = cla_new
    results_df["Language"] = lan
    

    accuracies_lan = {}
    for i in results_df["Language"].unique():
        lan_df = results_df.loc[results_df["Language"] == i]
        accuracies_lan[i] = accuracy_score(lan_df["True Label"], lan_df["Prediction"])
    accuracies_cla = {}
    for i in results_df["Classes_new"].unique():
        cla_df = results_df.loc[results_df["Classes_new"] == i]
        accuracies_cla[i] = accuracy_score(cla_df["True Label"], cla_df["Prediction"])
    accuracies_dat = {}
    for i in results_df["Dataset"].unique():
        dat_df = results_df.loc[results_df["Dataset"] == i]
        accuracies_dat[i] = accuracy_score(dat_df["True Label"], dat_df["Prediction"])
    accuracies_col = {}
    ind = []
    for i in range(results_df.shape[0]):
        if is_valid_word(results_df.iloc[i, 0]):
            ind.append(i)
    accuracies_col["valid_col_names"] = accuracy_score(results_df.iloc[ind, 2], results_df.iloc[ind, 1])
    accuracies_col["invalid_col_names"] = accuracy_score(results_df.iloc[[a for a in range(results_df.shape[0]) if a not in ind], 2], 
                                                        results_df.iloc[[a for a in range(results_df.shape[0]) if a not in ind], 1])

    accuracies_cla = dict(sorted(accuracies_cla.items(), key=lambda item: item[1], reverse=True))
    accuracies_lan = dict(sorted(accuracies_lan.items(), key=lambda item: item[1], reverse=True))
    accuracies_col = dict(sorted(accuracies_col.items(), key=lambda item: item[1], reverse=True))
    accuracies_dat = dict(sorted(accuracies_dat.items(), key=lambda item: item[1], reverse=True))


    fig = make_subplots(rows=3, cols=1, subplot_titles=["Accuracy per Class", "Number of correct predictions per Class", "Number of false predictions per Class"],
                        vertical_spacing=0.065, shared_xaxes=True)
    fig.add_trace(go.Bar(
        x=list(accuracies_cla.keys()),
        y=list(accuracies_cla.values()),
        marker=dict(color="black"),
        showlegend=False
    ))
    
    colors = [
    "#d62728",  # Red (High Saturation)
    "#ff9896",  # Red (Low Saturation)
    "#1f77b4",  # Blue (High Saturation)
    "#aec7e8",  # Blue (Low Saturation)
    "#ff7f0e",  # Orange (High Saturation)
    "#ffbb78",  # Orange (Low Saturation)
    "#2ca02c",  # Green (High Saturation)
    "#98df8a",  # Green (Low Saturation)
    "#9467bd",  # Purple (High Saturation)
    "#c5b0d5",  # Purple (Low Saturation)
    ]

    languages = ['english', 'french', 'german', 'mixed language', 'dessi data'] if dataset != "two_languages" else ["italian", "chinese"]
    for bool_val in [True, False]:
        for e, language in enumerate(languages):
            lan_df = results_df.loc[results_df["Language"] == language]
            accuracies_cla = {}
            for ee, col_valid in enumerate([True, False]):
                percentage_of_this_language = []
                for i in lan_df["Classes_new"].unique():
                    cla_df = lan_df.loc[lan_df["Classes_new"] == i]
                    ind = []
                    for j in range(cla_df.shape[0]):
                        if is_valid_word(cla_df.iloc[j, 0]) == col_valid:
                            ind.append(j)
                    cla_df = cla_df.iloc[ind,:]
                    percentage_of_this_language.append(cla_df.shape[0] / results_df.loc[results_df["Classes_new"] == i].shape[0])
                    accuracies_cla[i] = (cla_df["True Label"] == cla_df["Prediction"]).value_counts(normalize=True).get(bool_val, 0)
                fig.add_trace(go.Bar(
                    x=list(accuracies_cla.keys()),
                    y=[a * b for a, b in zip(list(accuracies_cla.values()), percentage_of_this_language)],
                    marker=dict(color=colors[2*e+ee]),
                    name=f"{language}_" + ("valid" if col_valid == True else "invalid") + "<br>column name",
                    showlegend=True if bool_val == True else False,
                    legendgroup=2*e+ee
                ), row=2 if bool_val == True else 3, col=1)
    fig.update_layout(title=f"Accuracy per Class for GPT's {title}", width=1500, height=700, barmode="stack")
    fig.update_yaxes(title_text="Accuracy", row=1, col=1)
    fig.update_yaxes(title_text="Amount of<br>correct predictions", row=2, col=1)
    fig.update_yaxes(title_text="Amount of<br>false predictions", range=[0,1.1], row=3, col=1)
    fig.show()

    fig = make_subplots(
        rows=1, cols=3, shared_yaxes=True,
        subplot_titles=["Accuracy per Language", "Accuracy per Dataset", "Accuracy per Column Name"]
    )
    fig.add_trace(go.Bar(
        x=list(accuracies_lan.keys()),
        y=list(accuracies_lan.values()),
        showlegend=False
    ), row=1, col=1)
    fig.add_trace(go.Bar(
        x=list(accuracies_dat.keys()),
        y=list(accuracies_dat.values()),
        showlegend=False
    ), row=1, col=2)
    fig.add_trace(go.Bar(
        x=list(accuracies_col.keys()),
        y=list(accuracies_col.values()),
        showlegend=False
    ), row=1, col=3)
    fig.update_layout(width=900, height=500, title=f"Accuracy of GPT's {title} with respect to different categories")
    fig.show()

In [10]:
create_analysis_plot(results_own_dataset_pii, "pii predictions on dessi-mf")

# Test two languages

In [11]:
predictions_test_languages, detected_classes_test_languages = load_predictions("gpt_predictions/test_languages_results.txt")

In [12]:
results_test_languages_pii = pd.DataFrame({
    "Column": test_lan.columns,
    "True Label": test_lan_pii["label"],
    "Prediction": predictions_test_languages,
    "Classes": test_lan_classes["class"],
    "Predicted Classes": detected_classes_test_languages,
    "Dataset": test_lan_datasets["dataset"]
})

In [13]:
plot_gpt_results(results_test_languages_pii, "GPT Prediction on test languages")

In [14]:
#False negatives
results_test_languages_pii.loc[(results_test_languages_pii["True Label"] == "pii") & (results_test_languages_pii["Prediction"] == "non-pii")]

,Column,True Label,Prediction,Classes,Predicted Classes,Dataset


All false negatives have corrupted column names and have except the address number values

In [15]:
#False positives
results_test_languages_pii.loc[(results_test_languages_pii["True Label"] == "non-pii") & (results_test_languages_pii["Prediction"] == "pii")]

,Column,True Label,Prediction,Classes,Predicted Classes,Dataset


In [16]:
create_analysis_plot(results_test_languages_pii, "pii predictions on test languages", dataset="two_languages")

# Kaggle Datasets

In [17]:
def load_predictions_dataset(path, data, labels, dataset):
    with open(path, "r") as f:
        response_text = f.read()
    responses = response_text.split("\n")[1:]
    pii = []
    for j in range(data.shape[1]):
        col = data.columns[j]
        col = col.replace("(", "\(").replace(")", "\)").replace("$", "\$")          # escape special characters for pattern matching
        match1 = re.search(f"({col}|'{col}'|\"{col}\")" +
                        f": (true|false|\"true\"|\"false\"|'true'|'false'|True|False|\"True\"|\"False\"|'True'|'False')",
                        responses[j])
        pii.append(match1.group(2))
    pii_transformed = ["non-pii" if "false" in a.lower() else "pii" for a in pii]
    
    results = pd.DataFrame({
        "Name": data.columns,
        "True Label": labels["label"],
        "Prediction": pii_transformed,
        "Dataset": dataset["dataset"]
    })
    return results

In [18]:
results_kaggle_pii = load_predictions_dataset("gpt_predictions/kaggle_results_pii.txt", kaggle_data, kaggle_labels, kaggle_dataset)

In [19]:
plot_gpt_results(results_kaggle_pii, title="GPT's prediction on Kaggle datasets")

In [20]:
#False negatives
results_kaggle_pii.loc[(results_kaggle_pii["True Label"] == "pii") & (results_kaggle_pii["Prediction"] == "non-pii")]


,Name,True Label,Prediction,Dataset
86,Serial No.,pii,non-pii,graduate_admissions
223,PassengerId,pii,non-pii,titanic
231,Ticket,pii,non-pii,titanic
233,Cabin,pii,non-pii,titanic


In [21]:
#False positives
results_kaggle_pii.loc[(results_kaggle_pii["True Label"] == "non-pii") & (results_kaggle_pii["Prediction"] == "pii")]

,Name,True Label,Prediction,Dataset
111,fullAddress,non-pii,pii,house_price
151,REGISTERED_OFFICE_ADDRESS,non-pii,pii,indian_companies
153,EMAIL_ADDR,non-pii,pii,indian_companies
179,director,non-pii,pii,pixar
180,writer,non-pii,pii,pixar
183,main_voice_actors,non-pii,pii,pixar


# OpenML Datasets

In [22]:
results_openml_pii = load_predictions_dataset("gpt_predictions/openml_results_pii.txt", openml_data, openml_labels, openml_dataset)

In [23]:
plot_gpt_results(results_openml_pii, title="GPT's prediction on OpenML datasets")

In [24]:
#False negatives
results_openml_pii.loc[(results_openml_pii["True Label"] == "pii") & (results_openml_pii["Prediction"] == "non-pii")]

,Name,True Label,Prediction,Dataset


In [25]:
#False positives
results_openml_pii.loc[(results_openml_pii["True Label"] == "non-pii") & (results_openml_pii["Prediction"] == "pii")]

,Name,True Label,Prediction,Dataset
4,Date of Birth,non-pii,pii,Amazon_Prime_Fiction
27,Referee,non-pii,pii,APL_20_24
86,Address,non-pii,pii,HousingPrices
105,customer_id,non-pii,pii,Oilst_Customers_Dataset
125,V16,non-pii,pii,TVS_Loan_Default


# OpenML 2 Datasets

In [26]:
results_openml_2_pii = load_predictions_dataset("gpt_predictions/openml_2_results_pii.txt", openml_2_data, openml_2_labels, openml_2_dataset)

In [27]:
plot_gpt_results(results_openml_2_pii, title="GPT's prediction on OpenML datasets")

In [28]:
#False negatives
results_openml_2_pii.loc[(results_openml_2_pii["True Label"] == "pii") & (results_openml_2_pii["Prediction"] == "non-pii")]

,Name,True Label,Prediction,Dataset


In [29]:
#False positives
results_openml_2_pii.loc[(results_openml_2_pii["True Label"] == "non-pii") & (results_openml_2_pii["Prediction"] == "pii")]

,Name,True Label,Prediction,Dataset
